In [2]:
import mlagents
import random
import numpy as np
from mlagents_envs.environment import UnityEnvironment as UE

In [3]:
env = UE(file_name="4444_project",seed=1,side_channels=[])
env.reset()

In [4]:
print(list(env.behavior_specs))
behavior_name = list(env.behavior_specs)[0]
print(behavior_name)

['CarBehavior?team=0']
CarBehavior?team=0


In [5]:
print(type(env.behavior_specs[behavior_name]))
print()
print(env.behavior_specs[behavior_name])
spec = env.behavior_specs[behavior_name]

<class 'mlagents_envs.base_env.BehaviorSpec'>

BehaviorSpec(observation_specs=[ObservationSpec(shape=(8,), dimension_property=(<DimensionProperty.NONE: 1>,), observation_type=<ObservationType.DEFAULT: 0>, name='VectorSensor_size8')], action_spec=ActionSpec(continuous_size=1, discrete_branches=()))


In [6]:
print(type(spec.observation_specs))
print()
print(type(tuple(spec.observation_specs[0])))
print(spec.observation_specs[0])

<class 'list'>

<class 'tuple'>
ObservationSpec(shape=(8,), dimension_property=(<DimensionProperty.NONE: 1>,), observation_type=<ObservationType.DEFAULT: 0>, name='VectorSensor_size8')


In [7]:
print(type(spec.action_spec))
print()
print(spec.action_spec)

<class 'mlagents_envs.base_env.ActionSpec'>

Continuous: 1, Discrete: ()


In [8]:
decision_steps, terminal_steps = env.get_steps(behavior_name)

In [9]:
print(decision_steps)
print(list(decision_steps))

[0]


In [10]:
tracked_agent = -1
done = False
episode_rewards = 0

print(type(decision_steps.agent_id))
tracked_agent = decision_steps.agent_id[0]

<class 'numpy.ndarray'>


In [11]:
action = spec.action_spec.random_action(len(decision_steps))
print(action)
print(type(action.discrete))
print(action.discrete)
# action = action.continuous

<class 'numpy.ndarray'>
[]


In [12]:
env.set_actions(behavior_name, action)

In [14]:
from dqn_brain import DQNAgent
from tqdm import tqdm

agent = DQNAgent()

for episode in tqdm(range(30)):
    env.reset()
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    current_state = decision_steps.obs[0].reshape(8,)
    tracked_agent = -1 # -1 indicates not yet tracking
    done = False # For the tracked_agent
    episode_rewards = 0 # For the tracked_agent

    agent.epsilon = agent.epsilon * agent.epsilon_decay

    while not done:
        # Track the first agent we see if not tracking
        # Note : len(decision_steps) = [number of agents that requested a decision]
        if tracked_agent == -1 and len(decision_steps) >= 1:
            tracked_agent = decision_steps.agent_id[0]

        # Generate an action for all agents
        # action = spec.action_spec.random_action(len(decision_steps))
        chosen_action_int = agent.select_action(current_state).item()
        # action = spec.action_spec.empty_action(len(decision_steps))
        action.add_continuous(np.array([[agent.index_to_action(chosen_action_int)]]))

        # Set the actions
        env.set_actions(behavior_name, action)
        # Move the simulation forward
        env.step()
        # Get the new simulation results
        decision_steps, terminal_steps = env.get_steps(behavior_name)
        print(decision_steps.obs)

        next_state = decision_steps.obs[0].reshape(8,)

        # print(decision_steps.obs)
        reward = 0
        if tracked_agent in decision_steps: # The agent requested a decision
            reward = decision_steps[tracked_agent].reward
            done = False
#             print(decision_steps[tracked_agent].reward)
        if tracked_agent in terminal_steps: # The agent terminated its episode
            reward = terminal_steps[tracked_agent].reward
#             print(terminal_steps[tracked_agent].reward)
            done = True
        episode_rewards += reward

        agent.memory.push((current_state, action, reward, next_state, done))
        agent.optimize_model()

    print(f"Total rewards for episode {episode} is {episode_rewards}")

  0%|          | 0/30 [00:00<?, ?it/s]

[array([[ 4.1899934, -0.3924   , 19.556175 , -1.       ,  7.623163 ,
         8.690312 , 11.358073 , 11.290971 ]], dtype=float32)]
[array([[ 2.1228652, -0.5886   , 19.887016 , -1.       ,  7.6976647,
         8.781966 , 12.933514 , 10.396062 ]], dtype=float32)]
[array([[ 0.03247886, -0.7848    , 19.999973  , 28.542267  ,  7.979289  ,
         8.929262  , 15.18194   ,  9.678803  ]], dtype=float32)]
[array([[-2.0582633, -0.981    , 19.893806 , -1.       ,  7.9153633,
         8.586748 , 13.134513 , 10.159246 ]], dtype=float32)]
[array([[ 0.03247885, -1.1772    , 19.999973  , 32.27882   ,  8.017331  ,
         8.572267  , 13.891187  ,  9.841566  ]], dtype=float32)]
[array([[-0.66553164, -1.3734    , 19.988922  , 29.496355  ,  8.144545  ,
         8.5539055 , 14.771752  ,  9.533302  ]], dtype=float32)]
[array([[-1.3627315, -1.5696   , 19.95352  , -1.       ,  8.093417 ,
         8.261795 , 12.899646 , 10.06879  ]], dtype=float32)]
[array([[ 0.73044264, -1.7658    , 19.986656  , 28.783821  

         9.362449 ,  8.107953 , 15.96344  ]], dtype=float32)]
[array([[ 9.881289  ,  0.35039955, 17.330824  , -1.        ,  6.674599  ,
         9.327091  ,  9.203701  , 13.793875  ]], dtype=float32)]
[array([[ 7.8710546,  0.3670001, 18.094313 , -1.       ,  6.641607 ,
         9.361879 ,  8.902037 , 14.395889 ]], dtype=float32)]
[array([[ 8.49504   ,  0.41821662, 17.759949  , -1.        ,  6.4555116 ,
         9.37071   ,  8.88387   , 13.896679  ]], dtype=float32)]
[array([[ 7.9498205 ,  0.37434775, 18.14194   , -1.        ,  6.5845327 ,
         9.48522   ,  8.3568    , 16.258387  ]], dtype=float32)]
[array([[ 9.728016 ,  0.518104 , 16.97764  , -1.       ,  6.4343596,
         9.544513 ,  8.240864 , 16.373348 ]], dtype=float32)]
[array([[ 9.8726845 ,  0.48750216, 17.150612  , -1.        ,  6.375293  ,
         9.663374  ,  7.966131  , 17.325508  ]], dtype=float32)]
[array([[10.414531 ,  0.5431042, 16.696585 , -1.       ,  6.308073 ,
         9.8106785,  7.696833 , 18.101404 ]], dtype

  3%|▎         | 1/30 [00:01<00:47,  1.63s/it]


[array([[ 0.      ,  0.      ,  0.      , -1.      ,  7.706062,  8.644832,
        10.248209, 12.426103]], dtype=float32)]
Total rewards for episode 0 is -4.950000073760748
[array([[ 4.1899934, -0.3924   , 19.556175 , -1.       ,  7.623163 ,
         8.690312 , 11.358073 , 11.290971 ]], dtype=float32)]
[array([[ 2.1228652, -0.5886   , 19.887016 , -1.       ,  7.6339607,
         8.65565  , 11.384626 , 11.243288 ]], dtype=float32)]
[array([[ 2.1228652, -0.7848   , 19.887016 , -1.       ,  7.6447873,
         8.620949 , 11.41123  , 11.195549 ]], dtype=float32)]
[array([[ 2.1228652, -0.981    , 19.887016 , -1.       ,  7.6556416,
         8.586209 , 11.437884 , 11.147753 ]], dtype=float32)]
[array([[ 2.1228652, -1.1772   , 19.887016 , -1.       ,  7.7318764,
         8.551433 , 11.464595 , 11.099899 ]], dtype=float32)]
[array([[ 2.1228652, -1.3734   , 19.887016 , -1.       ,  7.7468553,
         8.516619 , 11.491354 , 11.051988 ]], dtype=float32)]
[array([[ 2.1228652, -1.5696   , 19.8870

  3%|▎         | 1/30 [00:02<01:02,  2.16s/it]


[array([[ 4.116834  ,  0.09340023, 19.317686  , -1.        ,  7.7141533 ,
         8.304884  , 11.273649  , 11.489602  ]], dtype=float32)]
[array([[ 3.4226112,  0.1498695, 19.39212  , -1.       ,  7.7184916,
         8.32253  , -1.       , 13.567028 ]], dtype=float32)]
[array([[ 5.485211  ,  0.08728236, 19.064299  , -1.        ,  7.68555   ,
         8.336347  , 10.950943  , 13.089554  ]], dtype=float32)]
[array([[ 4.785292  ,  0.12843773, 19.121597  , -1.        ,  7.678856  ,
         8.374339  , 10.796016  , 13.652592  ]], dtype=float32)]
[array([[ 5.496711  ,  0.06100672, 19.067078  , -1.        ,  7.5651865 ,
         8.380248  , 10.9468975 , 12.260998  ]], dtype=float32)]
[array([[ 3.4757857e+00,  4.2470545e-03,  1.9524054e+01, -1.0000000e+00,
         7.5764956e+00,  8.4367571e+00,  1.2132944e+01,  1.1192526e+01]],
      dtype=float32)]
[array([[ 1.4150003e+00, -2.2869498e-02,  1.9745502e+01,  2.9311846e+01,
         7.7142100e+00,  8.5465784e+00,  1.4364538e+01,  9.7867527e+00

TypeError: an integer is required (got type ActionTuple)

In [16]:
env.reset()

In [29]:
decision_steps, terminal_steps = env.get_steps(behavior_name)
action = spec.action_spec.random_action(len(decision_steps))
print(action.discrete)
print(action.continuous)

[]
[[-0.3807651]]


In [30]:
decision_steps, terminal_steps = env.get_steps(behavior_name)
# action = spec.action_spec.random_action(len(decision_steps))
action = spec.action_spec.empty_action(len(decision_steps))
action.add_continuous(np.array([[0]]))
print(action.discrete)
print(action.continuous)

[]
[[0.]]


In [13]:
env.close()
